<a href="https://colab.research.google.com/github/qwebasilio/CPE019/blob/main/Assignment_7_1_Classifications_and_Regression_Basilio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hands-on Activity 5.2 | Build and Apply Multilayer Perceptron
--- | ---
**Name**  |  Basilio, Wilhelm Elian
**Course & Section**  |  CPE 019-CPE32S4
**Date** | March 26, 2025

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yasserh/wine-quality-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

df = pd.read_csv('/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv')
df['quality_category'] = pd.cut(df['quality'], bins=[0, 4, 6, 10], labels=[0, 1, 2])

X = df.drop(['quality', 'quality_category'], axis=1)
y = df['quality_category'].astype(int)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [8]:
def create_model():
    model = Sequential([
        Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(16, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = []

for train_idx, val_idx in kf.split(X_scaled):
    X_train_fold, X_val_fold = X_scaled[train_idx], X_scaled[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    model = create_model()
    model.fit(X_train_fold, y_train_fold, epochs=50, verbose=0, validation_data=(X_val_fold, y_val_fold))

    _, accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    scores.append(accuracy)

print("Average K-Fold Accuracy:", np.mean(scores))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Average K-Fold Accuracy: 0.8495058655738831


In [9]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, validation_split=0.2, verbose=0)

_, test_accuracy = model.evaluate(X_test, y_test)
print("Final Test Accuracy:", test_accuracy)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8508 - loss: 0.5102 
Final Test Accuracy: 0.8515284061431885


---

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

df = pd.read_csv('/root/.cache/kagglehub/datasets/yasserh/wine-quality-dataset/versions/1/WineQT.csv')

X = df.drop('quality', axis=1)
y = df['quality']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [11]:
def create_regression_model():
    model = Sequential([
        Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

base_model = create_regression_model()
base_model.fit(X_train, y_train, epochs=50, validation_split=0.2, verbose=0)

loss, mae = base_model.evaluate(X_test, y_test)
print("Base Model MAE:", mae)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6268 - mae: 0.6227 
Base Model MAE: 0.630454421043396


In [12]:
tuned_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])

tuned_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
tuned_model.fit(X_train, y_train, epochs=100, validation_split=0.2, verbose=0)

loss, mae = tuned_model.evaluate(X_test, y_test)
print("Tuned Model MAE:", mae)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5536 - mae: 0.5323 
Tuned Model MAE: 0.5282081961631775


---